# Appendix A.2: Computational Cost

This notebook reproduces the analyses in the Appendix, Section A.2 of the manuscript, including Tables 1-3.

> **Note:** By default, the computational cost experiments are run in parallel on 6 cores. If you have less cores available on your system, you should change that number according to your resources: `addprocs(5)` means that 5 processes on separates cores are added to the one being used already, so if you want to use, e.g., 4 cores, change that to `addprocs(3)`. If you do not want to run the code in parallel or have no sufficient resources to do to, you can comment out the line altogether and the code will run without further changes on a single core. 

## Setup 

First, we set up the parallel computing: 

In [6]:
using Distributed
addprocs(5)

5-element Vector{Int64}:
 2
 3
 4
 5
 6

Next, we load the Julia environment specified in the `Project.toml` and `Manifest.toml` files: First, we activate this environment, then install all dependencies (if some packages are not yet installed), and print out which packages and versions are currently in our environment. To make it available on all used processes, we use the `@everywhere` macro.

In [7]:
@everywhere using Pkg; 

# all paths are relative to the `notebook` subfolder main folder, i.e., assuming `pwd()` outputs
# ".../DeepDynamicodelingWithJust2TimePoints/notebooks"

@everywhere Pkg.activate("../.")
Pkg.instantiate()

      From worker 6:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 5:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 2:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 3:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 4:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`


  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`


Next, we load and precompile the necessary packages (in the versions specified by the `*.toml` files). 

In [8]:
@everywhere using BenchmarkTools
@everywhere using DataFrames
@everywhere using Distributed
@everywhere using Distributions
@everywhere using Random
@everywhere using Flux
@everywhere using DiffEqFlux
@everywhere using OrdinaryDiffEq
@everywhere using SharedArrays

Additionally, we import some user-defined functions, with different files for separate functionality, also using `@everywhere` to define them on all procs. 

In [9]:
@everywhere include("../src/simulation.jl") # for simulating data
@everywhere include("../src/model.jl") # for initializing and training the model
@everywhere include("../src/benchmarking.jl") # for plotting data and learned trajectories

## Define ground truth developments

First, we define the ground-truth developments as solutions of the underlying two-dimensional linear ODE system with two distinct sets of parameters, corresponding to two groups of individuals with two distinct underlying development patterns. 

In [10]:
# define initial condition
@everywhere true_u0 = Float32[2, 1]
# define time span on which to solve the ODE
@everywhere tspan = (0.0f0, 10.0f0)
# define parameters for the two distinct groups
@everywhere true_odeparams_group1 = Float32[-0.2, 0.00, 0.00, -0.2]
@everywhere true_odeparams_group2 = Float32[-0.2, 0.00, 0.00, 0.2]
  
# define corresponding ODE problems for the two groups
@everywhere prob1 = ODEProblem(linear_2d_system,true_u0,tspan,true_odeparams_group1)
@everywhere prob2 = ODEProblem(linear_2d_system,true_u0,tspan,true_odeparams_group2)
  
# solve ODE systems to obtain "true" underlying trajectory in each group
@everywhere dt=0.1
@everywhere sol_group1 = solve(prob1, Tsit5(), saveat = dt);
@everywhere sol_group2 = solve(prob2, Tsit5(), saveat = dt);

## Train model using benchmarking

Now, we train the model on varying numbers of individuals, time-dependent variables and baseline variables. We save all benchmark results, i.e., runtime, memory, and allocations, in a specific `SharedArray`, an array that allows for being used simultaneous by different processes while preventing them from getting in the way of each other. 

### Setup

In [11]:
# define number of observations, variables and baseline variables to try 
@everywhere n_obs = [50, 100, 250, 500, 1000, 2000, 5000]
@everywhere n_vars = [10, 20, 50, 100, 200]
@everywhere n_baselinevars = [10, 20, 50, 100, 200]
@everywhere lenobs, lenvars, lenbvars = length(n_obs), length(n_vars), length(n_baselinevars)

# construct dataframe: n, p, q, time, memory, allocations
benchmarkdf = DataFrame(n_obs = cat(n_obs, fill(100, lenvars + lenbvars), dims=1),
        n_vars = cat(fill(10, lenobs), n_vars, fill(10, lenbvars), dims=1),
        n_baselinevars = cat(fill(50, lenobs + lenvars), n_baselinevars, dims=1),
        time = fill(0.0, lenobs+lenvars+lenbvars),
        gctime = fill(0.0, lenobs+lenvars+lenbvars),
        memory = fill(0, lenobs+lenvars+lenbvars),
        allocs = fill(0, lenobs+lenvars+lenbvars)
)

# get it to a shared array for distributed computing
benchmarkarray = SharedArrays.SharedMatrix{Float64}(size(Matrix(benchmarkdf)));
benchmarkarray[:,1:3] = Matrix(benchmarkdf)[:,1:3];

@everywhere eval($benchmarkarray);

### Scenario 1: Fixed number of time-dep and baseline variables, varying number of observations 

In [8]:
@sync @distributed for n_ind in 1:lenobs
    # warmup (first run takes longer because of precompilation times and shouldnt be included)
    n_warmup, p_warmup, q_warmup, q_info_warmup = 100, 10, 10, 10
    xs, x_baseline, tvals, group1, group2 = generate_all(n_warmup, p_warmup, q_warmup, q_info_warmup);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p_warmup, q_warmup, zdim, nODEparams, prob1)
    L = loss_wrapper(m)
    ps = getparams(m)
    opt = ADAM(0.0005)
    for epoch in 1:35
        Flux.train!(L, ps, trainingdata, opt)
    end
    println("warmup done")
    # now start for real
    n, p, q = n_obs[n_ind], 10, 50 
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = n_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 3:	warmup done
      From worker 3:	n=250, p=10, q=50
      From worker 2:	warmup done
      From worker 2:	n=50, p=10, q=50
      From worker 4:	warmup done
      From worker 4:	n=1000, p=10, q=50
      From worker 6:	warmup done
      From worker 6:	n=5000, p=10, q=50
      From worker 5:	warmup done
      From worker 5:	n=2000, p=10, q=50
      From worker 2:	training done
      From worker 2:	warmup done
      From worker 2:	n=100, p=10, q=50
      From worker 2:	training done
      From worker 3:	training done
      From worker 3:	warmup done
      From worker 3:	n=500, p=10, q=50
      From worker 3:	training done
      From worker 4:	training done
      From worker 5:	training done
      From worker 6:	training done


Task (done) @0x000000014dc45ee0

### Scenario 2: Fixed number of observations and baseline variables, varying number of time-dependent variables

In [9]:
@sync @distributed for p_ind in 1:lenvars
    n, p, q = 100, n_vars[p_ind], 50 
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = lenobs + p_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 4:	n=100, p=50, q=50
      From worker 3:	n=100, p=20, q=50
      From worker 2:	n=100, p=10, q=50
      From worker 6:	n=100, p=200, q=50
      From worker 5:	n=100, p=100, q=50
      From worker 2:	training done
      From worker 3:	training done
      From worker 4:	training done
      From worker 5:	training done
      From worker 6:	training done


Task (done) @0x0000000164834ab0

### Scenario 3: Fixed number of observations and time-dependent variables, varying number of baseline variables

In [10]:
@sync @distributed for q_ind in 1:lenbvars
    n, p, q = 100, 10, n_baselinevars[q_ind]
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = lenobs + lenvars + q_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 3:	n=100, p=10, q=20
      From worker 4:	n=100, p=10, q=50
      From worker 2:	n=100, p=10, q=10
      From worker 5:	n=100, p=10, q=100
      From worker 6:	n=100, p=10, q=200
      From worker 2:	training done
      From worker 3:	training done
      From worker 4:	training done
      From worker 5:	training done
      From worker 6:	training done


Task (done) @0x0000000164836540

## Save results 

First, we can optionally save the Julia object as `JLD2` file:

In [13]:
# if desired: save as JLD2 file 
using JLD2 
JLD2.@save "../benchmarkresults.jld2" benchmarkarray
# and re-load from saved
JLD2.@load "../benchmarkresults.jld2" 
benchmarkarray = eval(:benchmarkarray)

17×7 SharedMatrix{Float64}:
   50.0   10.0   50.0  6.23103e9   2.12793e8   1.69195e9   1.74255e7
  100.0   10.0   50.0  1.35203e10  4.8314e8    3.38413e9   3.48681e7
  250.0   10.0   50.0  3.02995e10  1.16346e9   8.46084e9   8.72037e7
  500.0   10.0   50.0  5.19168e10  2.06915e9   1.6927e10   1.74473e8
 1000.0   10.0   50.0  9.91186e10  4.13274e9   3.38457e10  3.48972e8
 2000.0   10.0   50.0  1.69727e11  7.6441e9    6.76913e10  6.97943e8
 5000.0   10.0   50.0  4.1173e11   2.01611e10  1.69239e11  1.74553e9
  100.0   10.0   50.0  1.47972e10  4.79092e8   3.38414e9   3.48681e7
  100.0   20.0   50.0  1.60899e10  5.41337e8   3.92201e9   4.46942e7
  100.0   50.0   50.0  2.03554e10  7.62936e8   5.93587e9   7.40913e7
  100.0  100.0   50.0  2.83108e10  2.19809e9   1.06392e10  1.23093e8
  100.0  200.0   50.0  5.67905e10  1.10356e10  2.50541e10  2.21111e8
  100.0   10.0   10.0  1.43175e10  5.28021e8   3.30699e9   3.4867e7
  100.0   10.0   20.0  1.424e10    5.35328e8   3.31774e9   3.486e7
  100.0  

Now, we copy back the information from the `SharedArray` object to the benchmark dataframe, to export that to CSV format.

In [14]:
# copy back to dataframe, to be saved later as CSV
benchmarkdf[:,:time] = benchmarkarray[:,4]
benchmarkdf[:,:gctime] = benchmarkarray[:,5]
benchmarkdf[:,:memory] = benchmarkarray[:,6]
benchmarkdf[:,:allocs] = benchmarkarray[:,7]
benchmarkdf

,n_obs,n_vars,n_baselinevars,time,gctime,memory,allocs
,Int64,Int64,Int64,Float64,Float64,Int64,Int64
1,50,10,50,6.23103e9,2.12793e8,1691949568,17425528
2,100,10,50,1.35203e10,4.8314e8,3384134912,34868112
3,250,10,50,3.02995e10,1.16346e9,8460841120,87203666
4,500,10,50,5.19168e10,2.06915e9,16927000176,174473248
5,1000,10,50,9.91186e10,4.13274e9,33845661856,348971551
6,2000,10,50,1.69727e11,7.6441e9,67691281696,697942793
7,5000,10,50,4.1173e11,2.01611e10,169238884896,1745528013
8,100,10,50,1.47972e10,4.79092e8,3384143088,34868119
9,100,20,50,1.60899e10,5.41337e8,3922006848,44694194


Additionally, we turn the time and memory information into human-readable format and units: 

In [15]:
benchmarkdf[:,:time_in_seconds] = round.(benchmarkarray[:,4] .* 1e-9, digits=3)

# turn memory into human-readable format (taken from BenchmarkTools.jl source code)
benchmarkdf[:,:prettymemory] = prettymemory.(benchmarkarray[:,6])
benchmarkdf

,n_obs,n_vars,n_baselinevars,time,gctime,memory,allocs
,Int64,Int64,Int64,Float64,Float64,Int64,Int64
1,50,10,50,6.23103e9,2.12793e8,1691949568,17425528
2,100,10,50,1.35203e10,4.8314e8,3384134912,34868112
3,250,10,50,3.02995e10,1.16346e9,8460841120,87203666
4,500,10,50,5.19168e10,2.06915e9,16927000176,174473248
5,1000,10,50,9.91186e10,4.13274e9,33845661856,348971551
6,2000,10,50,1.69727e11,7.6441e9,67691281696,697942793
7,5000,10,50,4.1173e11,2.01611e10,169238884896,1745528013
8,100,10,50,1.47972e10,4.79092e8,3384143088,34868119
9,100,20,50,1.60899e10,5.41337e8,3922006848,44694194


Finally, we can export to CSV; re-creating Tables 1-3 from the manuscript.

In [16]:
# save entire dataframe as CSV
using CSV 
CSV.write("benchmarkresults.csv", benchmarkdf)

# extract tables as in manuscript appendix and save as CSV files

# different number of observations for fixed p (10) and q (50)
benchmark_obs = benchmarkdf[1:7, [:n_obs, :time_in_seconds, :prettymemory]]
@show(benchmark_obs)

benchmark_vars = benchmarkdf[8:12, [:n_vars, :time_in_seconds, :prettymemory]]
@show(benchmark_obs)

benchmark_bvars = benchmarkdf[13:end, [:n_baselinevars, :time_in_seconds, :prettymemory]];
@show(benchmark_bvars)

benchmark_obs = 7×3 DataFrame
 Row │ n_obs  time_in_seconds  prettymemory
     │ Int64  Float64          String
─────┼──────────────────────────────────────
   1 │    50            6.231  1.58 GiB
   2 │   100           13.52   3.15 GiB
   3 │   250           30.3    7.88 GiB
   4 │   500           51.917  15.76 GiB
   5 │  1000           99.119  31.52 GiB
   6 │  2000          169.727  63.04 GiB
   7 │  5000          411.73   157.62 GiB
benchmark_obs = 7×3 DataFrame
 Row │ n_obs  time_in_seconds  prettymemory
     │ Int64  Float64          String
─────┼──────────────────────────────────────
   1 │    50            6.231  1.58 GiB
   2 │   100           13.52   3.15 GiB
   3 │   250           30.3    7.88 GiB
   4 │   500           51.917  15.76 GiB
   5 │  1000           99.119  31.52 GiB
   6 │  2000          169.727  63.04 GiB
   7 │  5000          411.73   157.62 GiB
benchmark_bvars = 5×3 DataFrame
 Row │ n_baselinevars  time_in_seconds  prettymemory
     │ Int64           Float64 

,n_baselinevars,time_in_seconds,prettymemory
,Int64,Float64,String
1,10,14.317,3.08 GiB
2,20,14.24,3.09 GiB
3,50,14.387,3.15 GiB
4,100,14.459,3.36 GiB
5,200,14.859,4.17 GiB


We can now save the dataframes individualls as `.csv` files. 

In [16]:
CSV.write("../benchmarkresults_baselinevars.csv", benchmark_bvars)
CSV.write("../benchmarkresults_obs.csv", benchmark_obs)
CSV.write("../benchmarkresults_vars.csv", benchmark_vars)

"benchmarkresults_vars.csv"